In [2]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
pdf_docs = PyPDFLoader("../00_data/Sustainability_report_2024_kr.pdf").load()  
# PDF 파일을 불러와 각 페이지를 LangChain의 Document 형태로 로드  
# 각 페이지는 'page_content'(본문 텍스트)와 'metadata'(페이지 번호 등 정보)를 포함

len(pdf_docs)  
# 불러온 PDF 문서의 페이지 수를 확인

83

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
rec_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunk_docs = rec_splitter.split_documents(pdf_docs)
len(chunk_docs)

207

In [6]:
chunk_docs[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')

In [ ]:
for item in chunk_docs:
    item.metadata = {**(item.metadata), "class" : "wanted"}  # metadata에 class : wanted 키와 벨류를 추가함

In [8]:
chunk_docs[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')

### Fiass 벡터 DB 생성

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings  
# OpenAI의 임베딩(벡터화) 기능을 LangChain에서 사용하기 위한 클래스 불러오기

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")  
# OpenAI의 'text-embedding-3-large' 모델을 사용해 텍스트를 벡터로 변환하는 객체 생성

In [ ]:
dim_size = len(embeddings.embed_query("안녕하세요"))  
# "안녕하세요"라는 문장을 임베딩(벡터화)하여 생성된 벡터의 길이를 계산
# 즉, 벡터가 몇 차원(dim)인지 확인

print(dim_size)  
# 임베딩 벡터의 차원 수를 출력

3072


In [11]:
# uv add faiss-cpu

In [12]:
chunk_docs[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')

In [ ]:
import faiss  
# Facebook AI에서 개발한 고속 벡터 검색 라이브러리  
# 임베딩 벡터를 효율적으로 저장하고, 유사도 검색을 빠르게 수행할 수 있음

from langchain_community.vectorstores import FAISS  
# LangChain에서 FAISS 기반 벡터 저장소를 쉽게 사용하기 위한 래퍼 클래스  
# 임베딩 벡터를 DB처럼 저장하고 검색할 수 있음

from langchain_community.docstore.in_memory import InMemoryDocstore  
# 벡터와 연결된 문서(metadata)를 메모리에 저장하는 간단한 문서 저장소  
# 검색 시 벡터와 원문 문서를 함께 조회할 때 사용

In [ ]:
db = FAISS.from_documents(
    documents = [chunk_docs[0]],  # 벡터 DB에 저장할 문서 리스트. 여기서는 첫 번째 청크(chunk)만 사용
    embedding = embeddings,       # 문서를 벡터로 변환할 때 사용할 OpenAI 임베딩 객체
    # ids = ["문서1"]  문서 이름도 임의로 설정 가능
)
# FAISS 벡터 저장소를 생성하고, 주어진 문서를 임베딩하여 저장
# 이후 검색(query) 시 벡터를 기반으로 유사 문서를 빠르게 조회 가능

In [ ]:
db.index_to_docstore_id  
# FAISS 벡터 저장소에서 벡터(index)와 실제 문서(Document) ID 간의 매핑 정보를 확인
# key: FAISS 내부 벡터 인덱스 번호
# value: 해당 벡터가 연결된 Document 객체의 ID
# 검색 결과에서 벡터를 찾아 원문 문서를 가져올 때 사용됨

{0: '753a9bef-3c76-4e77-9b4c-d0243369e0b0'}

In [16]:
db = FAISS.from_documents(
    documents = [chunk_docs[0]],
    embedding = embeddings,
)
db.index_to_docstore_id

{0: '2a42c148-49dc-4c23-8d39-141aa76e8a43'}

In [ ]:
db.docstore.__dict__["_dict"]  
# FAISS 벡터 저장소와 연결된 InMemoryDocstore 안의 실제 문서 저장소 확인
# _dict: 내부적으로 Document ID를 key로, Document 객체를 value로 저장한 딕셔너리
# 즉, 벡터와 매칭된 원본 문서 내용을 직접 들여다볼 수 있는 구조

{'2a42c148-49dc-4c23-8d39-141aa76e8a43': Document(id='2a42c148-49dc-4c23-8d39-141aa76e8a43', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')}

In [ ]:
db.similarity_search("삼성", k=5)  
# 벡터 저장소(db)에서 "삼성"이라는 질문(query)과 가장 유사한 문서 청크 상위 5개(k=5)를 검색
# 반환값: 검색된 Document 객체 리스트

[Document(id='2a42c148-49dc-4c23-8d39-141aa76e8a43', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')]

In [ ]:
vectorstore_db_path = "../07_vectorstore/samsung_faiss.db"  # 벡터 DB를 저장할 로컬 경로 지정
index_name = "samsung2025"                                  # 저장할 인덱스(데이터베이스) 이름 설정

db.save_local(
    folder_path = vectorstore_db_path,  # 지정한 폴더 경로에
    index_name = index_name             # 해당 이름으로 벡터 DB 저장
)

In [ ]:
# 저장된 db 불러오기
load_db = FAISS.load_local(
    folder_path = vectorstore_db_path,         # 이전에 저장해둔 FAISS 벡터 DB 폴더 경로
    index_name = index_name,                   # 불러올 인덱스(저장된 벡터 파일) 이름
    embeddings = embeddings,                   # 동일한 임베딩 모델로 벡터를 복원하기 위해 지정
    allow_dangerous_deserialization = True     # 로컬에 저장된 pickle 데이터를 복원 허용 (주의 필요)
)


In [23]:
load_db.docstore.__dict__["_dict"]

{'2a42c148-49dc-4c23-8d39-141aa76e8a43': Document(id='2a42c148-49dc-4c23-8d39-141aa76e8a43', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 0, 'page_label': '1', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024')}

### 문서 추가하기

In [24]:
chunk_docs[1:10]

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 1, 'page_label': '2', 'class': 'wanted'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024\nCEO 메시지\n회사 소개\n이해관계자 소통\nOur Company\n04\n05\n06\n준법과 윤리경영\nPrinciple\n53\n중대성 평가\nMateriality Assessment\n08\n임직원\n공급망\n사회공헌\n개인정보보호/보안\n고객의 안전/품질\nPeople\n31\n39\n45\n48\n50\n경제성과\n사회성과\n환경성과\n지역별 수자원 현황   \n사업부문별 환경성과\nFacts & Figures\n56\n57\n62\n65\n66\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서\nScope 3 온실가스 배출량 검증 의견서\nGRI Index\nTCFD 대조표\nSASB 대조표\n전사차원의 기후변화 대응 협력 활동\nAbout This Report\nAppendix\n70\n71\n72\n74\n77\n79\n81\n82\n[DX부문] \n추진체계 및 주요성과\n기후변화\n자원순환\n수자원 및 오염물질\n[DS부문]  \n추진체계 및 주요성과 \n기후변화\n수자원\n폐기물\n오염물질\nPlanet\n12\n13\n15\n17\n19\n20\n23\n26\n28\n삼성전자 지속

In [ ]:
load_db.add_documents(  # 문서 추가
    chunk_docs[1:10]
)

['f2fdcd3e-d4b4-42a6-8cfd-f9bd1563556a',
 '9d99530b-af18-4e32-b5f0-2482d53c7938',
 'ff0fcd06-3b1d-4759-abc2-d9ad991b3a4c',
 '7195fd7c-41de-4c05-8d6d-1ebf0b94912a',
 '0130340e-90e7-4824-9128-17412845d4b6',
 'a2ba209d-693c-4fde-9c57-c89aef4ad03f',
 'a0b18591-229b-454f-9f55-83b9b364c6ee',
 'aa782d75-5291-4553-9e40-a620cd86e7fb',
 '3b3f1b11-3541-443c-8978-dd908b6562d1']

In [26]:
load_db.index_to_docstore_id

{0: '2a42c148-49dc-4c23-8d39-141aa76e8a43',
 1: 'f2fdcd3e-d4b4-42a6-8cfd-f9bd1563556a',
 2: '9d99530b-af18-4e32-b5f0-2482d53c7938',
 3: 'ff0fcd06-3b1d-4759-abc2-d9ad991b3a4c',
 4: '7195fd7c-41de-4c05-8d6d-1ebf0b94912a',
 5: '0130340e-90e7-4824-9128-17412845d4b6',
 6: 'a2ba209d-693c-4fde-9c57-c89aef4ad03f',
 7: 'a0b18591-229b-454f-9f55-83b9b364c6ee',
 8: 'aa782d75-5291-4553-9e40-a620cd86e7fb',
 9: '3b3f1b11-3541-443c-8978-dd908b6562d1'}

In [27]:
vectorstore_db_path = "../07_vectorstore/samsung_faiss.db"
index_name = "samsung2025"
load_db.save_local(
    folder_path = vectorstore_db_path,
    index_name = index_name
)

In [28]:
# 저장된 db 불러오기
updated_db = FAISS.load_local(
    folder_path = vectorstore_db_path,
    index_name = index_name,
    embeddings = embeddings,
    allow_dangerous_deserialization = True
)
updated_db.index_to_docstore_id

{0: '2a42c148-49dc-4c23-8d39-141aa76e8a43',
 1: 'f2fdcd3e-d4b4-42a6-8cfd-f9bd1563556a',
 2: '9d99530b-af18-4e32-b5f0-2482d53c7938',
 3: 'ff0fcd06-3b1d-4759-abc2-d9ad991b3a4c',
 4: '7195fd7c-41de-4c05-8d6d-1ebf0b94912a',
 5: '0130340e-90e7-4824-9128-17412845d4b6',
 6: 'a2ba209d-693c-4fde-9c57-c89aef4ad03f',
 7: 'a0b18591-229b-454f-9f55-83b9b364c6ee',
 8: 'aa782d75-5291-4553-9e40-a620cd86e7fb',
 9: '3b3f1b11-3541-443c-8978-dd908b6562d1'}

In [29]:
chunk_docs[2]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 2, 'page_label': '3', 'class': 'wanted'}, page_content='Our Company\n04    CEO 메시지\n05    회사 소개\xa0\n06    이해관계자 소통\n삼성전자 지속가능경영보고서 2024 03Our Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People')

In [ ]:
from langchain_core.documents import Document   # LangChain에서 문서(Document) 객체를 사용하기 위해 임포트

# 기존 벡터 DB(updated_db)에 새 문서 추가
updated_db.add_documents(
    [
        Document(                                # 첫 번째 문서 객체 생성
            page_content = "새로운 문서는 이렇게 추가하기",  # 실제 문서 내용 (텍스트)
            metadata = {"source" : "수동"}                # 메타데이터 (출처 정보)
        ),
        Document(                                # 두 번째 문서 객체 생성
            page_content = "2024년 삼성 전자 주식 사지마세요",  # 실제 문서 내용
            metadata = {"source" : "윤택한"}                # 메타데이터 (작성자 또는 출처)
        )
    ]
)

['3c12c578-0596-45d5-97f2-fc49884eb64c',
 '948c42aa-b55e-4477-b072-6b583fcac8c7']

In [31]:
updated_db.index_to_docstore_id

{0: '2a42c148-49dc-4c23-8d39-141aa76e8a43',
 1: 'f2fdcd3e-d4b4-42a6-8cfd-f9bd1563556a',
 2: '9d99530b-af18-4e32-b5f0-2482d53c7938',
 3: 'ff0fcd06-3b1d-4759-abc2-d9ad991b3a4c',
 4: '7195fd7c-41de-4c05-8d6d-1ebf0b94912a',
 5: '0130340e-90e7-4824-9128-17412845d4b6',
 6: 'a2ba209d-693c-4fde-9c57-c89aef4ad03f',
 7: 'a0b18591-229b-454f-9f55-83b9b364c6ee',
 8: 'aa782d75-5291-4553-9e40-a620cd86e7fb',
 9: '3b3f1b11-3541-443c-8978-dd908b6562d1',
 10: '3c12c578-0596-45d5-97f2-fc49884eb64c',
 11: '948c42aa-b55e-4477-b072-6b583fcac8c7'}

In [ ]:
updated_db.delete(['3c12c578-0596-45d5-97f2-fc49884eb64c']) # 삭제하는 코드
updated_db.index_to_docstore_id

{0: '2a42c148-49dc-4c23-8d39-141aa76e8a43',
 1: 'f2fdcd3e-d4b4-42a6-8cfd-f9bd1563556a',
 2: '9d99530b-af18-4e32-b5f0-2482d53c7938',
 3: 'ff0fcd06-3b1d-4759-abc2-d9ad991b3a4c',
 4: '7195fd7c-41de-4c05-8d6d-1ebf0b94912a',
 5: '0130340e-90e7-4824-9128-17412845d4b6',
 6: 'a2ba209d-693c-4fde-9c57-c89aef4ad03f',
 7: 'a0b18591-229b-454f-9f55-83b9b364c6ee',
 8: 'aa782d75-5291-4553-9e40-a620cd86e7fb',
 9: '3b3f1b11-3541-443c-8978-dd908b6562d1',
 10: '948c42aa-b55e-4477-b072-6b583fcac8c7'}

In [ ]:
updated_db.similarity_search("삼성", k=5)
# "삼성"이라는 검색어를 벡터로 변환해 updated_db(업데이트된 벡터 DB)에 질의
# 의미적으로 가장 유사한 문서 5개(k=5)를 검색하여 반환
# 반환값은 Document 객체 리스트이며, 각 객체에는 텍스트(page_content)와 메타데이터(metadata)가 포함됨

[Document(id='948c42aa-b55e-4477-b072-6b583fcac8c7', metadata={'source': '윤택한'}, page_content='2024년 삼성 전자 주식 사지마세요'),
 Document(id='9d99530b-af18-4e32-b5f0-2482d53c7938', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 2, 'page_label': '3', 'class': 'wanted'}, page_content='Our Company\n04    CEO 메시지\n05    회사 소개\xa0\n06    이해관계자 소통\n삼성전자 지속가능경영보고서 2024 03Our Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People'),
 Document(id='2a42c148-49dc-4c23-8d39-141aa76e8a43', metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_p

### 벡터 스토어 합치기

- 물리적 합치기
- 검색기만 하이브리드로 사용

In [34]:
# 2024년 데이터를 앞에서 10개 -> db1
# 2024년 데이터를 11~20 -> db2
# 이 두 벡터 스토어를 합쳐서 -> db3

In [ ]:
db1 = FAISS.from_documents(    # 첫 번째 FAISS 벡터 데이터베이스 생성
    chunk_docs[:10],           # chunk_docs 리스트의 앞부분(0~9번째 문서 10개)만 사용
    embedding = embeddings     # 문서를 벡터로 변환할 임베딩 모델 지정
)

db2 = FAISS.from_documents(    # 두 번째 FAISS 벡터 데이터베이스 생성
    chunk_docs[10:20],         # chunk_docs 리스트의 다음 부분(10~19번째 문서 10개) 사용
    embedding = embeddings     # 동일한 임베딩 모델 사용
)


In [36]:
# 1. 새로운 공간에 합친 데이터베이스를 만들기 -> db3
# db1 에 db2 를 합쳐버리기

In [ ]:
db1.merge_from(     # 두 개의 벡터 데이터베이스를 합치는 메서드
    target = db2    # db2의 내용을 db1에 병합 (db1 ← db2)
)

db1.index_to_docstore_id   # 병합된 db1의 내부 인덱스와 문서 ID 매핑 관계 확인

{0: 'cb5cc75b-a3ee-4594-86ed-83409f591bbe',
 1: '2a6ff3f1-102b-411b-ad22-be5bc4f1728c',
 2: '2c4ba470-0944-47d3-97d8-52299ee03e8e',
 3: '71489837-7c5d-49e4-97f2-8858e8b1caa9',
 4: '97c2561b-8d9b-4a75-8735-a74783d22714',
 5: 'a580577e-9ec6-48bf-9a0b-f0694a293537',
 6: '0d4a051d-f022-4b5a-893c-dfa97229f519',
 7: '3def8bbd-1b3d-4265-b095-3001bf2f36cd',
 8: '7e4a6035-309f-41b4-b232-df630174a3eb',
 9: '64244995-3a4f-438e-a35b-26b3cbc04be3',
 10: '48177335-76b9-4a51-b2ac-cb980324ae2a',
 11: 'dfb86749-0d4e-4360-be8c-822a1c8bc18e',
 12: '8ee833f6-1c3e-4347-b914-8fd4ed1cad36',
 13: 'a2652702-d08f-4331-9ae1-6c615de3b136',
 14: '1bb0e3c5-edaf-418b-958e-0194bee66409',
 15: '49ec62a9-1f07-4410-8d24-c72c5f5b9e95',
 16: 'af3ef049-a5a9-437b-bc71-527df550887e',
 17: 'd2f6a238-f695-41d5-82c4-f6872cdc2622',
 18: '3f1bbce7-e9c7-4485-be88-aa8719cf02fb',
 19: '0a19ee06-09bb-4374-8fe7-2a6ff4778c66'}

In [38]:
db2.index_to_docstore_id

{0: '48177335-76b9-4a51-b2ac-cb980324ae2a',
 1: 'dfb86749-0d4e-4360-be8c-822a1c8bc18e',
 2: '8ee833f6-1c3e-4347-b914-8fd4ed1cad36',
 3: 'a2652702-d08f-4331-9ae1-6c615de3b136',
 4: '1bb0e3c5-edaf-418b-958e-0194bee66409',
 5: '49ec62a9-1f07-4410-8d24-c72c5f5b9e95',
 6: 'af3ef049-a5a9-437b-bc71-527df550887e',
 7: 'd2f6a238-f695-41d5-82c4-f6872cdc2622',
 8: '3f1bbce7-e9c7-4485-be88-aa8719cf02fb',
 9: '0a19ee06-09bb-4374-8fe7-2a6ff4778c66'}

In [ ]:
# 만약에 나는 전혀 다른 db3 에다가 만들고 싶어
db3 = FAISS(                                # 새로운 FAISS 벡터 데이터베이스 객체 생성
    docstore = InMemoryDocstore(),          # 메모리 기반 문서 저장소 (문서 내용 저장용)
    index_to_docstore_id = {},              # 인덱스 번호와 문서 ID를 매핑할 빈 딕셔너리
    embedding_function = embeddings,        # 텍스트를 벡터로 변환할 임베딩 모델 지정
    index = faiss.IndexFlatL2(dim_size)     # L2 거리(유클리드 거리) 기반의 벡터 검색 인덱스 생성
)

In [40]:
db3.merge_from(
    target=db2
)
db2.index_to_docstore_id

{0: '48177335-76b9-4a51-b2ac-cb980324ae2a',
 1: 'dfb86749-0d4e-4360-be8c-822a1c8bc18e',
 2: '8ee833f6-1c3e-4347-b914-8fd4ed1cad36',
 3: 'a2652702-d08f-4331-9ae1-6c615de3b136',
 4: '1bb0e3c5-edaf-418b-958e-0194bee66409',
 5: '49ec62a9-1f07-4410-8d24-c72c5f5b9e95',
 6: 'af3ef049-a5a9-437b-bc71-527df550887e',
 7: 'd2f6a238-f695-41d5-82c4-f6872cdc2622',
 8: '3f1bbce7-e9c7-4485-be88-aa8719cf02fb',
 9: '0a19ee06-09bb-4374-8fe7-2a6ff4778c66'}

In [ ]:
"samsung2025::5aa19b24-ee08-419f-a766-8666d44a3d9c"